In [16]:
import pandas as pd
import numpy as np
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from surprise import Reader, Dataset, SVD

import warnings; warnings.simplefilter('ignore')

In [17]:
df = pd.read_csv("movies_features.csv")
df.head(5)

,movieId,title,directors,writers,cast,genres
0,862,Toy Story,['johnlasseter'],"['josswhedon', 'andrewstanton', 'joelcohen', '...","['tomhanks', 'timallen', 'donrickles', 'jimvar...","['animation', 'comedy', 'family']"
1,8844,Jumanji,['joejohnston'],"['jonathanhensleigh', 'gregtaylor', 'jimstrain']","['robinwilliams', 'jonathanhyde', 'kirstenduns...","['adventure', 'fantasy', 'family']"
2,15602,Grumpier Old Men,['howarddeutch'],['markstevenjohnson'],"['waltermatthau', 'jacklemmon', 'ann-margret',...","['romance', 'comedy']"
3,31357,Waiting to Exhale,['forestwhitaker'],"['ronaldbass', 'terrymcmillan']","['whitneyhouston', 'angelabassett', 'lorettade...","['comedy', 'drama', 'romance']"
4,11862,Father of the Bride Part II,['charlesshyer'],"['nancymeyers', 'alberthackett']","['stevemartin', 'dianekeaton', 'martinshort', ...",['comedy']


Trying with first 3 of writers and cast

In [20]:
def extract_first_three(lst):
    return lst[:3]

# apply the function to the desired column
df['writers'] = df['writers'].apply(extract_first_three)
df['cast'] = df['cast'].apply(extract_first_three)

# print the result
df.head(5)

,movieId,title,directors,writers,cast,genres
0,862,Toy Story,['johnlasseter'],['j,['t,"['animation', 'comedy', 'family']"
1,8844,Jumanji,['joejohnston'],['j,['r,"['adventure', 'fantasy', 'family']"
2,15602,Grumpier Old Men,['howarddeutch'],['m,['w,"['romance', 'comedy']"
3,31357,Waiting to Exhale,['forestwhitaker'],['r,['w,"['comedy', 'drama', 'romance']"
4,11862,Father of the Bride Part II,['charlesshyer'],['n,['s,['comedy']


In [11]:
df['soup'] = df['directors'] + df['writers'] + df['cast'] + df['genres']

In [12]:
df.head(5)

,movieId,title,directors,writers,cast,genres,soup
0,862,Toy Story,['johnlasseter'],"['josswhedon', 'andrewstanton', 'joelcohen', '...","['tomhanks', 'timallen', 'donrickles', 'jimvar...","['animation', 'comedy', 'family']","['johnlasseter']['josswhedon', 'andrewstanton'..."
1,8844,Jumanji,['joejohnston'],"['jonathanhensleigh', 'gregtaylor', 'jimstrain']","['robinwilliams', 'jonathanhyde', 'kirstenduns...","['adventure', 'fantasy', 'family']","['joejohnston']['jonathanhensleigh', 'gregtayl..."
2,15602,Grumpier Old Men,['howarddeutch'],['markstevenjohnson'],"['waltermatthau', 'jacklemmon', 'ann-margret',...","['romance', 'comedy']",['howarddeutch']['markstevenjohnson']['walterm...
3,31357,Waiting to Exhale,['forestwhitaker'],"['ronaldbass', 'terrymcmillan']","['whitneyhouston', 'angelabassett', 'lorettade...","['comedy', 'drama', 'romance']","['forestwhitaker']['ronaldbass', 'terrymcmilla..."
4,11862,Father of the Bride Part II,['charlesshyer'],"['nancymeyers', 'alberthackett']","['stevemartin', 'dianekeaton', 'martinshort', ...",['comedy'],"['charlesshyer']['nancymeyers', 'alberthackett..."


In [13]:
count = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
count_matrix = count.fit_transform(df['soup'])

In [14]:
def get_recommendations(title):
    idx = indices[title]
    if type(idx) != np.int64:
        if len(idx)>1:
            print("ALERT: Multiple values")
            idx = idx[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

cosine_sim = cosine_similarity(count_matrix, count_matrix)
df = df.reset_index()
titles = df['title']
indices = pd.Series(df.index, index=df['title'])

get_recommendations('Mean Girls').head(10)

15483                                  American Fork
22569                                      Ο Ξεναγός
40627     The First Annual 'On Cinema' Oscar Special
42795    The Fourth Annual 'On Cinema' Oscar Special
43951                                   World Cinema
44912                         The Roller Skate Craze
44925                              Le Cochon Danseur
45171                               Puolin ja toisin
33799                            Live from New York!
29660                                 Monkey Kingdom
Name: title, dtype: object